In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
names=[]
# Assign column names to the dataset
for i in range(1,769):
 names.append("feature_"+str(i))
labels=["label_1","label_2","label_3","label_4"]
names+=labels

# Read in the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_project/DataSet_L7/train.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_project/DataSet_L7/valid.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Mini_project/DataSet_L7/test.csv')

In [4]:
from sklearn.preprocessing import RobustScaler

x_train ={}
y_train ={}
x_valid ={}
y_valid ={}
x_test = {}

df_t = train_df
df_v = valid_df

for label in labels:
  scaler = RobustScaler()
  if label == 'label_2':
    df_t = train_df.dropna()
    df_v = valid_df.dropna()

  x_train[label] = scaler.fit_transform(df_t.drop(labels, axis=1))
  y_train[label] = df_t[label]
  x_valid[label] = scaler.transform(df_v.drop(labels, axis=1))
  y_valid[label] = df_v[label]
  x_test[label] = scaler.transform(test_df.drop(['ID'], axis=1))




In [ ]:
pd.DataFrame(x_train['label_2']).shape

(28040, 768)

#Label 3

##PCA

In [5]:
from sklearn.decomposition import PCA

# Instantiate PCA with a desired number of components (e.g., n_components=50)
pca = PCA(n_components=0.97)

# Fit PCA on your training data
x_train_pca = pca.fit_transform(x_train['label_3'])

# Transform your validation and test data using the same PCA model
x_valid_pca = pca.transform(x_valid['label_3'])
x_test_pca = pca.transform(x_test['label_3'])  # If you have a test set

# Now, you can use x_train_pca, x_valid_pca, and x_test_pca as your reduced-dimensional feature vectors for modeling.
x_train_pca.shape

(28040, 411)

##KNN

###Original


In [6]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train['label_3'], y_train['label_3'])

KNeighborsClassifier()

In [7]:
y_pred = classifier.predict(x_valid['label_3'])

In [8]:
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
print(confusion_matrix(y_valid['label_3'], y_pred))
print(accuracy_score(y_valid['label_3'], y_pred))

[[137   5]
 [  6 588]]
0.9850543478260869


###After PCA

In [9]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x_train_pca, y_train['label_3'])

y_pred = classifier.predict(x_valid_pca)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_valid['label_3'], y_pred))
print(accuracy_score(y_valid['label_3'], y_pred))

[[139   3]
 [  6 588]]
0.9877717391304348


###After PCA + HP tuning

##SVM

###Original

In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import uniform, randint

In [11]:
svm_classifier = SVC(class_weight='balanced' ,C=1, kernel='rbf')
svm_classifier.fit(x_train['label_3'], y_train['label_3'])

# Make predictions on the test data
y_pred = svm_classifier.predict(x_valid['label_3'])

# Evaluate model performance
print(accuracy_score(y_valid['label_3'], y_pred))

0.998641304347826


###After PCA

In [12]:
svm_classifier = SVC(class_weight='balanced' ,C=1, kernel='rbf')
svm_classifier.fit(x_train_pca, y_train['label_3'])

# Make predictions on the test data
y_pred = svm_classifier.predict(x_valid_pca)

# Evaluate model performance
print(accuracy_score(y_valid['label_3'], y_pred))

0.998641304347826


In [13]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Define the cross-validation strategy (e.g., StratifiedKFold with 5 folds)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define your SVM classifier with the desired hyperparameters
svm_classifier = SVC(class_weight='balanced', C=1, kernel='rbf')

# Perform cross-validation and calculate accuracy scores
scores = cross_val_score(svm_classifier, x_train['label_3'], y_train['label_3'], cv=cv, scoring='accuracy')

# Print the cross-validation scores and mean accuracy
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())


Cross-validation scores: [0.99696862 0.9978602  0.99875178 0.99857347 0.9989301 ]
Mean accuracy: 0.9982168330955778


###After PCA + HP tuning

In [ ]:
pd.DataFrame(x_train_pca)

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,14.141137,1.757399,-6.011598,-1.549809,-5.533205,3.069755,-1.930802,2.005719,0.429892,6.139024,...,0.193010,-0.492481,-0.073585,0.365000,0.035715,0.018748,0.013538,0.030854,-0.135038,-0.412747
1,-0.055825,7.098817,2.441451,6.707103,4.784069,-3.199298,1.051284,0.670880,-2.809372,1.371238,...,-0.053823,0.081717,0.594274,0.086277,-0.355933,0.562878,-0.079062,0.016058,0.818525,0.022031
2,-4.181035,11.751734,-5.137172,0.876813,-4.564641,0.979505,2.597162,-0.002129,0.177368,-2.895701,...,-0.096682,-0.186332,-0.076901,-0.164443,0.120631,-0.303517,0.104136,0.106510,-0.035545,0.164923
3,-1.618872,-0.421247,6.360317,0.293023,-1.193303,-4.980271,1.333557,7.554658,4.333935,1.598901,...,-0.399560,-0.210579,-0.116511,0.107600,-0.010901,-0.690007,-0.005226,0.162164,0.016159,0.525058
4,2.740203,1.552950,-2.641521,5.782143,10.572141,1.117830,4.666824,-1.300311,6.340427,-3.190307,...,-0.194258,0.158061,-0.390869,-0.304014,-0.082834,0.104290,0.271381,-0.098731,-0.493513,-0.065833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28515,-5.651805,13.804510,-3.727424,-0.470373,-9.544083,1.284753,3.092439,-0.683669,-1.009392,-1.890243,...,0.016002,0.213209,0.020319,0.036019,0.026550,0.337825,-0.203154,0.292060,-0.077269,-0.149470
28516,0.656379,0.127061,-6.609875,-7.931759,4.223875,-3.498405,-0.578157,2.649631,-3.184036,-4.518412,...,-0.267557,0.198904,-0.125069,0.204384,0.340939,-0.144584,0.261425,-0.115345,0.000215,-0.188724
28517,-7.730342,2.395539,3.669455,-3.432518,1.333108,9.413259,-2.096879,1.174211,-0.654335,1.907130,...,0.011813,0.526027,-0.251539,0.244195,0.155801,0.004485,0.174467,0.004056,-0.110188,0.133053
28518,12.988838,-0.759326,-5.440311,-1.970844,-3.861694,5.732011,-2.475813,1.967593,0.976162,2.886242,...,0.083468,-0.060319,-0.272793,0.278703,-0.117891,0.266656,0.060700,0.112991,-0.045264,0.477330


In [28]:
# svm_classifier = SVC(class_weight='balanced' ,C=1, degree=4, gamma='auto', kernel='linear') #0.9918478
svm_classifier = SVC(class_weight='balanced' ,C=50 ,kernel='rbf') #0.998641
#Best Hyperparameters: {'C': 30.46237691733707, 'degree': 1, 'kernel': 'poly'} =0.99
svm_classifier.fit(x_train_pca, y_train['label_3'])

# Make predictions on the test data
y_pred = svm_classifier.predict(x_valid_pca)

# Evaluate model performance
print(accuracy_score(y_valid['label_3'], y_pred))

0.998641304347826


In [ ]:
# Define the hyperparameter grid
param_grid = {
    'C': [0.01, 0.1, 1,10,100],  # Adjust the range
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Experiment with different kernels
    'class_weight': ['balanced',None],  # Change class_weight options
    'gamma':['scale', 'auto'],
    'degree': [1,2,3,4]

}

# Create an SVM classifier
svm_classifier = SVC()

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm_classifier, param_grid, scoring='accuracy', cv=5,n_jobs=-1)
grid_search.fit(x_train_pca, y_train['label_1'])

# Get the best model and hyperparameters
best_svm_classifier = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions on the validation set
y_pred = best_svm_classifier.predict(x_valid_pca)

# Evaluate the model
print("Best Hyperparameters:", best_params)
print("Classification Report on Validation Data:")
print(classification_report(y_valid['label_1'], y_pred))

In [ ]:
param_distributions = {
    'C': uniform(0.001,100),  # Continuous uniform distribution for 'C'
    'kernel': ['linear','poly'],  # Experiment with different kernels
    # 'class_weight': ['balanced', None],  # Change class_weight options
    # 'gamma':['scale', 'auto'],
    'degree': [1,2,3,4]
}

# Create an SVM classifier
svm_classifier = SVC()

# Example with Randomized Search
random_search = RandomizedSearchCV(svm_classifier, param_distributions, n_iter=20, cv=5, scoring='accuracy', random_state=42)
random_search.fit(x_train_pca, y_train['label_3'])


# Get the best hyperparameters and model
best_svm_classifier = random_search.best_estimator_
best_params = random_search.best_params_

# Make predictions on the validation data
y_pred = best_svm_classifier.predict(x_valid_pca)

# Evaluate the model
print("Best Hyperparameters:", best_params)
print("Classification Report on Validation Data:")
print(classification_report(y_valid['label_3'], y_pred))

Best Hyperparameters: {'C': 30.46237691733707, 'degree': 1, 'kernel': 'poly'}
Classification Report on Validation Data:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       142
           1       1.00      0.99      1.00       594

    accuracy                           0.99       736
   macro avg       0.99      0.99      0.99       736
weighted avg       0.99      0.99      0.99       736



In [ ]:
label_1_pred = svm_classifier.predict(x_test_pca)

##Random Forest

###Orginal

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier  # You can replace this with any other model


In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train['label_1'], y_train['label_1'])  # Assuming x_train and y_train are your feature and target data


RandomForestClassifier()

In [ ]:
# Make predictions on the test data
y_pred = model.predict(x_valid['label_1'])

# Evaluate model performance
print(classification_report(y_valid['label_1'], y_pred))

              precision    recall  f1-score   support

           1       0.83      0.77      0.80        13
           2       0.89      0.89      0.89         9
           3       0.79      0.92      0.85        12
           4       0.89      1.00      0.94        16
           5       0.89      0.89      0.89        18
           6       0.90      1.00      0.95         9
           7       0.84      0.94      0.89        17
           8       1.00      0.79      0.88        14
           9       0.90      0.82      0.86        11
          10       0.89      1.00      0.94         8
          11       0.90      0.95      0.92        19
          12       0.88      1.00      0.93         7
          13       0.91      0.91      0.91        11
          14       0.88      0.93      0.90        15
          15       0.89      0.94      0.91        17
          16       0.87      0.93      0.90        14
          17       0.87      0.93      0.90        14
          18       0.92    

###After PCA

###After PCA + HP tuning

##XGBoost

In [ ]:
pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Initialize the XGBoost classifier
model = xgb.XGBClassifier()
y_train_shifted = y_train['label_1'] - 1

# Train the model
model.fit(x_train['label_1'],y_train_shifted )




KeyboardInterrupt: ignored

In [ ]:
y_pred = model.predict(x_valid['label_1'])

accuracy = accuracy_score(y_valid['label_1'], y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Additional evaluation metrics
print(classification_report(y_valid['label_1'], y_pred))


##Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Initialize the Decision Tree classifier
model = DecisionTreeClassifier(random_state=42)

# Train the model
model.fit(x_train['label_1'], y_train['label_1'])


DecisionTreeClassifier(random_state=42)

In [ ]:
y_pred = model.predict(x_valid['label_1'])


In [ ]:
accuracy = accuracy_score(y_valid['label_1'], y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Additional evaluation metrics
print(classification_report(y_valid['label_1'], y_pred))


Accuracy: 0.37
              precision    recall  f1-score   support

           1       0.27      0.31      0.29        13
           2       0.33      0.44      0.38         9
           3       0.40      0.33      0.36        12
           4       0.38      0.38      0.38        16
           5       0.42      0.28      0.33        18
           6       0.27      0.33      0.30         9
           7       0.50      0.35      0.41        17
           8       0.33      0.29      0.31        14
           9       0.55      0.55      0.55        11
          10       0.71      0.62      0.67         8
          11       0.69      0.47      0.56        19
          12       0.67      0.86      0.75         7
          13       0.19      0.27      0.22        11
          14       0.40      0.40      0.40        15
          15       0.45      0.53      0.49        17
          16       0.30      0.21      0.25        14
          17       0.62      0.57      0.59        14
          18